In [15]:
import numpy as np
import math
import time
import resource
visited = []
stack=[]

In [2]:
def goal_test(state):
  for row in range(state.shape[0]):
    for col in range(state.shape[1]):
      if state[row][col] == "0":
        return False
  return True

In [3]:
def find_agent_position(maze):
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i][j] == "S":
                return i, j

In [4]:
def check_adjacent_tiles_for_zeroes(maze, row, col):
    # check the tile to the left
    if col > 0 and maze[row][col-1] == '0':
        return True
    # check the tile to the right
    if col < len(maze[0])-1 and maze[row][col+1] == '0':
        return True
    # check the tile above
    if row > 0 and maze[row-1][col] == '0':
        return True
    # check the tile below
    if row < len(maze)-1 and maze[row+1][col] == '0':
        return True
    
    return False


In [5]:
def get_valid_directions(maze,row,col,visited):
    valid_directions = []
    if row > 0 and maze[row-1][col] != 'X':
        valid_directions.append('up')
    if row < maze.shape[0]-1 and maze[row+1][col] != 'X':
        valid_directions.append('down')
    if col > 0 and maze[row][col-1] != 'X':
        valid_directions.append('left')
    if col < maze.shape[1]-1 and maze[row][col+1] != 'X':
        valid_directions.append('right')
    
    return valid_directions

In [53]:
def get_direction_cost(maze, routes):
    costs = {}    
    endpoint1=None
    endpoint2=None
    endpoint3=None
    endpoint4=None
    
    if not routes:
      a="Pop"
      costs[a]=0
    for direction in routes:
        step_cost = 0
        agent_row, agent_col = find_agent_position(maze)
        if direction == "right":
            while maze[agent_row][agent_col+1] != "X":
                agent_col += 1
                step_cost += 1
            c,r= agent_row,agent_col
            endpoint1= maze[c][r]
            stack.append((agent_row,agent_col))
        elif direction == "left":
            while maze[agent_row][agent_col-1] != "X":
                agent_col -= 1
                step_cost += 1
            stack.append((agent_row,agent_col))
            c,r= agent_row,agent_col
            endpoint2= maze[c][r]
        elif direction == "up":
            while maze[agent_row-1][agent_col] != "X":
                agent_row -= 1
                step_cost += 1
            stack.append((agent_row,agent_col))
            c,r= agent_row,agent_col
            endpoint3= maze[c][r]
        elif direction == "down":
            while maze[agent_row+1][agent_col] != "X":
                agent_row += 1
                step_cost += 1
            stack.append((agent_row,agent_col))
            c,r= agent_row,agent_col
            endpoint4= maze[c][r]
        endpoints = [endpoint1, endpoint2, endpoint3, endpoint4]
        zero_count = endpoints.count("0")
        if zero_count == 1:
            zero_index = endpoints.index("0")
            costs[direction] = step_cost
            return costs
        else:
          costs[direction] = step_cost
          
    return costs

In [7]:
def find_nearest_zero(maze, row, col):
    nearest_zero_dist = float('inf')
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == '0':
                dist = abs(i - row) + abs(j - col)  # calculate Manhattan distance
                if dist < nearest_zero_dist:
                    nearest_zero_dist = dist
    return nearest_zero_dist


In [8]:
import copy

def get_direction_costs_a(amaze, routes):
    maze = copy.deepcopy(amaze)
    costs = {}   
    total_costs = {} 
    if not routes:
        a = "Pop"
        costs[a] = 0
    for direction in routes:
        step_cost = 0
        total = 0
        agent_row, agent_col = find_agent_position(maze)
        if direction == "right":
            while maze[agent_row][agent_col+1] != "X":
                agent_col += 1
                step_cost += 1
                maze[agent_row][agent_col] = "1"
                if goal_test(maze):
                    costs[direction]=step_cost
                    total_costs[direction]=total
                    return costs,total_costs
                    
            cost_a = find_nearest_zero(maze, agent_row, agent_col)
            total = cost_a + step_cost
        elif direction == "left": 
            while maze[agent_row][agent_col-1] != "X":
                agent_col -= 1
                step_cost += 1
                maze[agent_row][agent_col] = "1"
                if goal_test(maze):
                    costs[direction]=step_cost
                    total_costs[direction]=total
                    return costs,total_costs
            cost_a = find_nearest_zero(maze, agent_row, agent_col)
            total = cost_a + step_cost
        elif direction == "up":
            while maze[agent_row-1][agent_col] != "X":
                agent_row -= 1
                step_cost += 1
                maze[agent_row][agent_col] = "1"
                if goal_test(maze):
                    costs[direction]=step_cost
                    total_costs[direction]=total
                    return costs,total_costs
            cost_a = find_nearest_zero(maze, agent_row, agent_col)
            total = cost_a + step_cost
        elif direction == "down":
            while maze[agent_row+1][agent_col] != "X":
                agent_row += 1
                step_cost += 1
                maze[agent_row][agent_col] = "1"
                if goal_test(maze):
                    costs[direction]=step_cost
                    total_costs[direction]=total
                    return costs,total_costs
            cost_a = find_nearest_zero(maze, agent_row, agent_col)
            total = cost_a + step_cost
        total_costs[direction] = total
        costs[direction] = step_cost      
    return costs, total_costs


In [9]:
import random

def get_least_cost_route(costs, prev_direction=None):
    # Check if there is only one valid direction
    if len(costs) == 1:
        return list(costs.keys())[0], list(costs.values())[0]
    
    # Choose the direction with the least cost
    min_key = min(costs, key=costs.get)
    
    # If there are multiple directions with the same cost, choose a random one
    min_keys = [k for k in costs if costs[k] == costs[min_key]]
    min_key = random.choice(min_keys)
        
    if prev_direction is not None and len(costs) > 1:
        # Check that the chosen direction is not the opposite of the previous direction
        opposite_directions = {'up': 'down', 'down': 'up', 'left': 'right', 'right': 'left'}
        if min_key == opposite_directions[prev_direction]:
            # If the chosen direction is opposite to the previous direction, remove it from the options
            del costs[min_key]
            return get_least_cost_route(costs, prev_direction)
            
    return min_key, costs[min_key]


In [10]:
import random

def get_least_cost_route_a(costs, prev_direction=None):
    # Check if there is only one valid direction
    if len(costs) == 1:
        return list(costs.keys())[0], list(costs.values())[0]
    
    # Choose the direction with the least cost
    min_key = min(costs, key=costs.get)
    
    # If there are multiple directions with the same cost, choose a random one
    min_keys = [k for k in costs if costs[k] == costs[min_key]]
    min_key = random.choice(min_keys)
        
    if prev_direction is not None and len(costs) > 1:
        # Check that the chosen direction is not the opposite of the previous direction
        opposite_directions = {'up': 'down', 'down': 'up', 'left': 'right', 'right': 'left'}
        if min_key == opposite_directions[prev_direction]:
            # If the chosen direction is opposite to the previous direction, remove it from the options
            del costs[min_key]
            return get_least_cost_route(costs, prev_direction)
            
    return min_key, costs[min_key]


In [11]:
def move_agent(maze, visited, stack):
    total_cost = 0
    prev_route=None
    print("Starting Maze:")
    print(maze)
    while not goal_test(maze):
        agent_pos = find_agent_position(maze)
        row, col = agent_pos
        stack.append((row, col))
        print("\nCurrent Position: ({},{})".format(row, col))
        routes = get_valid_directions(maze, row, col, visited)
        print("Valid Directions: ", routes)
        if not routes:  # No valid directions available
          print("Error: Agent is trapped, cannot move to any direction!")
          break
        # Get the cost of each direction
        costs = get_direction_cost(maze, routes)
        print("Costs: ", costs)
              
        # Choose the cheapest route
        route, cost = get_least_cost_route(costs,prev_route)
        print("Cheap Route: ", route, "Route Costs: ", cost)
        prev_route=route
        if route == 'right':
          maze[row][col] = '1'
          for i in range(cost):
            maze[row][col+i+1] = '1'
            visited.append((row, col+i+1))
          col += cost
          maze[row][col] = 'S'
        elif route == 'left':
          maze[row][col] = '1'

          for i in range(cost):
            maze[row][col-i-1] = '1'             
            visited.append((row, col-i-1))
          col -= cost
          maze[row][col] = 'S'
        
        elif route == 'down':
          maze[row][col] = '1'

          for i in range(cost):
            maze[row+i+1][col] = '1'
            visited.append((row+i+1, col))
          row += cost
          maze[row][col] = 'S'

        elif route == 'up':
          maze[row][col] = '1'

          for i in range(cost):
            maze[row-i-1][col] = '1'
            visited.append((row-i-1, col))
          row -= cost
          maze[row][col] = 'S'        
          
        elif route == "Pop":
          a, b = row, col
          if len(stack) > 0:
              row, col = stack.pop()
          c, d = row, col
          total_cost += abs(c - a) + abs(d - b)
          maze[a][b] = "1"
          maze[row][col] = "S"
              
        # Increment the total cost
        total_cost += costs[route]
        print(maze)
          
    return maze, total_cost, visited, stack     


In [12]:
def move_agent_a(maze, visited, stack):
    total_cost = 0
    print("Starting Maze:")
    print(maze)
    prev_route=None
    while not goal_test(maze):
        agent_pos = find_agent_position(maze)
        row, col = agent_pos
        stack.append((row, col))
        print("\nCurrent Position: ({},{})".format(row, col))
        routes = get_valid_directions(maze, row, col, visited)
        print("Valid Directions: ", routes)
        if not routes:  # No valid directions available
          print("Error: Agent is trapped, cannot move to any direction!")
          break
        if len(routes) > 1:
            costs, heuristic_costs = get_direction_costs_a(maze, routes)
            print("Costs: ", costs)
            print("Costs with Heuristics: ", heuristic_costs)
            route,heuristic_cost= get_least_cost_route_a(heuristic_costs,prev_route)
            cost=costs[route]
            # If we are not at a decision node, continue like usual taking the only path available.
        else:
            costs = get_direction_cost(maze, routes)
            print("Costs: ", costs)
            route, cost = get_least_cost_route(costs)

        print("Cheap Route: ", route, "Route Costs: ", cost)
        prev_route=route
        if route == 'right':
            maze[row][col] = '1'
            for i in range(cost):
                maze[row][col+(i+1)] = '1'
                visited.append((row, col+(i+1)))
            stack.append((row, col+cost))
            col += cost
            maze[row][col] = 'S'
        elif route == 'left':
            maze[row][col] = '1'
            for i in range(cost):
                maze[row][col-(i+1)] = '1'             
                visited.append((row, col-(i+1)))
            stack.append((row, col-cost))
            col -= cost
            maze[row][col] = 'S'
        elif route == 'down':
            maze[row][col] = '1'
            for i in range(cost):
                maze[row+(i+1)][col] = '1'
                visited.append((row+(i+1), col))
            stack.append((row+cost, col))
            row += cost
            maze[row][col] = 'S'
        elif route == 'up':
            maze[row][col] = '1'
            for i in range(cost):
                maze[row-(i+1)][col] = '1'
                visited.append((row-(i+1), col))
            stack.append((row-cost, col))
            row -= cost
            maze[row][col] = 'S'
        elif route == "Pop":
            maze[row][col] = "1"
            a, b = row, col
            if len(stack) > 0:
                stack.pop()
                e = stack.pop()
                print()
                maze[row][col] = "S"
                c, d = row, col
                total_cost += abs(c - a) + abs(d - b)
        total_cost += costs.get(route, 0)
        print(maze)
        print(stack)

    return maze, total_cost, visited, stack


In [38]:
def count_zeros(maze):
    count = np.count_nonzero(maze == "0")
    return count

In [99]:
maze15 = np.array([    
    ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
    ["X", "0", "0", "X", "0", "0", "X", "X", "X", "X", "X"],
    ["X", "0", "0", "0", "0", "0", "0", "0", "0", "X", "X"],
    ["X", "X", "0", "X", "0", "X", "0", "X", "0", "X", "X"],
    ["X", "0", "0", "X", "0", "X", "0", "X", "0", "0", "X"],
    ["X", "0", "X", "X", "0", "X", "0", "X", "X", "X", "X"],
    ["X", "0", "0", "X", "S", "X", "0", "X", "0", "0", "X"],
    ["X", "X", "0", "X", "X", "X", "0", "0", "0", "0", "X"],
    ["X", "0", "0", "0", "0", "0", "0", "0", "0", "X", "X"],
    ["X", "0", "0", "X", "X", "X", "X", "X", "X", "X", "X"],
    ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
])



total_cost=0
visited = []
stack=[]


start_time = time.time()
maze=maze15
nozeros=count_zeros(maze)
# Start measuring CPU time and memory usage
start_cpu_time = resource.getrusage(resource.RUSAGE_SELF).ru_utime
start_memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
maze,total_cost,visited,stack=move_agent_a(maze,visited,stack)
#maze,total_cost,visited,stack=move_agent(maze,visited,stack)

end_time = time.time()

# End measuring CPU time and memory usage
end_cpu_time = resource.getrusage(resource.RUSAGE_SELF).ru_utime
end_memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

# Calculate time and CPU consumption
elapsed_time = end_time - start_time
elapsed_cpu_time = end_cpu_time - start_cpu_time

print("The total cost= ", total_cost)
print("Wall-clock time:", elapsed_time)
print("CPU time:", elapsed_cpu_time)
print("Memory usage:", end_memory_usage)
print("Total nodes expanded:", len(visited))
print("Total zeros:", nozeros)

Starting Maze:
[['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' '0' '0' 'X' '0' '0' 'X' 'X' 'X' 'X' 'X']
 ['X' '0' '0' '0' '0' '0' '0' '0' '0' 'X' 'X']
 ['X' 'X' '0' 'X' '0' 'X' '0' 'X' '0' 'X' 'X']
 ['X' '0' '0' 'X' '0' 'X' '0' 'X' '0' '0' 'X']
 ['X' '0' 'X' 'X' '0' 'X' '0' 'X' 'X' 'X' 'X']
 ['X' '0' '0' 'X' 'S' 'X' '0' 'X' '0' '0' 'X']
 ['X' 'X' '0' 'X' 'X' 'X' '0' '0' '0' '0' 'X']
 ['X' '0' '0' '0' '0' '0' '0' '0' '0' 'X' 'X']
 ['X' '0' '0' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']]

Current Position: (6,4)
Valid Directions:  ['up']
Costs:  {'up': 5}
Cheap Route:  up Route Costs:  5
[['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' '0' '0' 'X' 'S' '0' 'X' 'X' 'X' 'X' 'X']
 ['X' '0' '0' '0' '1' '0' '0' '0' '0' 'X' 'X']
 ['X' 'X' '0' 'X' '1' 'X' '0' 'X' '0' 'X' 'X']
 ['X' '0' '0' 'X' '1' 'X' '0' 'X' '0' '0' 'X']
 ['X' '0' 'X' 'X' '1' 'X' '0' 'X' 'X' 'X' 'X']
 ['X' '0' '0' 'X' '1' 'X' '0' 'X' '0' '0' 'X']
 ['X' 'X' '0' 'X' 'X' 'X' '0' '0' '0